In [1]:
%cd /root/mobilenet_v2_ssdlite_keras

/root/mobilenet_v2_ssdlite_keras


In [2]:
import sys
sys.path.append('/root/mobilenet_v2_ssdlite_keras')

In [3]:
from __future__ import division
import os
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from losses.keras_ssd_loss import SSDLoss
from utils.object_detection_2d_data_generator import DataGenerator
from utils.object_detection_2d_geometric_ops import Resize
from utils.object_detection_2d_photometric_ops import ConvertTo3Channels
from utils.data_augmentation_chain_original_ssd import SSDDataAugmentation
from utils.coco import get_coco_category_maps
from utils.ssd_input_encoder import SSDInputEncoder
# from models.keras_mobilenet_v2_ssdlite import mobilenet_v2_ssd
from layers.AnchorBoxesLayer import AnchorBoxes
from layers.DecodeDetectionsLayer import DecodeDetections
from layers.DecodeDetectionsFastLayer import DecodeDetectionsFast
from models.graphs.mobilenet_v2_ssdlite_praph import mobilenet_v2_ssdlite
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import backend as K 
from tensorflow.keras.layers import Input, Lambda, Activation, Conv2D, \
    DepthwiseConv2D, Reshape, Concatenate, BatchNormalization, ReLU
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger
from glob import glob
from tensorflow import keras

2021-12-11 08:50:51.225770: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
K.clear_session()

In [5]:
# model config
batch_size = 128
image_size = (256, 256, 3)
n_classes = 4
mode = 'training'
l2_regularization = 0.0005
min_scale = 0.1
max_scale = 0.9
scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05]
scales = scales_pascal
aspect_ratios_global = None
aspect_ratios  = [[1.0, 2.0, 0.5], 
                [1.0, 2.0, 0.5, 3.0, 1.0 / 3.0], 
                [1.0, 2.0, 0.5, 3.0, 1.0 / 3.0],
                [1.0, 2.0, 0.5, 3.0, 1.0 / 3.0], 
                [1.0, 2.0, 0.5], 
                [1.0, 2.0, 0.5]]
two_boxes_for_ar1 = True
steps = None
offsets = None
clip_boxes = False
variances = [0.1, 0.1, 0.2, 0.2]

In [6]:
path = "/root/mobilenet_v2_ssdlite_keras/checkpoint.h5"

model =load_model(
   path, custom_objects={"AnchorBoxes":AnchorBoxes},  compile = False)

2021-12-11 08:51:03.659851: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-11 08:51:03.661421: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-11 08:51:04.184742: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 08:51:04.185619: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2021-12-11 08:51:04.185673: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-12-11 08:51:04.190387: I tensorflow/stream_executor/platform/defau

In [7]:
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

In [8]:
model.compile(optimizer=adam, loss=ssd_loss.compute_loss, metrics = ['accuracy'])

In [9]:
# 데이터 로드
train_dataset = DataGenerator()
val_dataset = DataGenerator()

In [10]:
train_images_dir = '/root/OIDv4_ToolKit/OID/Dataset/train_all/'
train_annotations_dir = '/root/OIDv4_ToolKit/OID/Dataset/train_annotation/'
train_image_set_filename = '/root/OIDv4_ToolKit/OID/Dataset/train.txt'


test_images_dir = '/root/OIDv4_ToolKit/OID/Dataset/test_all/'
test_annotations_dir = '/root/OIDv4_ToolKit/OID/Dataset/test_annotation/'
test_image_set_filename     = '/root/OIDv4_ToolKit/OID/Dataset/test.txt'


validation_images_dir = '/root/OIDv4_ToolKit/OID/Dataset/validation_all/'
validation_annotations_dir = '/root/OIDv4_ToolKit/OID/Dataset/validation_annotation/'
validation_image_set_filename     = '/root/OIDv4_ToolKit/OID/Dataset/validation.txt'

In [11]:
classes=['Hat', 'Helmet', 'Bicycle helmet', 'Human head']

In [12]:
train_dataset.parse_xml(
                        images_dirs = [train_images_dir],
                        image_set_filenames = [train_image_set_filename],
                        annotations_dirs = [train_annotations_dir],
                        classes = classes,
                        include_classes = 'all',
                        exclude_truncated = False,
                        exclude_difficult = False,
                        ret = False
                        )


val_dataset.parse_xml(images_dirs = [validation_images_dir],
                      image_set_filenames = [validation_image_set_filename],
                      annotations_dirs = [validation_annotations_dir],
                      classes=classes,
                      include_classes='all',
                      exclude_truncated=False,
                      exclude_difficult=False,#used to be True
                      ret=False)

Processing image set 'validation.txt': 100%|████████████████████████████████████████████████████| 3530/3530 [00:06<00:00, 530.93it/s]


In [13]:
img_height = 256 # Height of the model input images
img_width = 256 # Width of the model input images
img_channels = 3 #

In [14]:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width)

In [15]:
# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.
#用getlayer来获取输出层的尺寸
# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('ssd_cls1conv2_bn').output_shape[1:3],
                   model.get_layer('ssd_cls2conv2_bn').output_shape[1:3],
                   model.get_layer('ssd_cls3conv2_bn').output_shape[1:3],
                   model.get_layer('ssd_cls4conv2_bn').output_shape[1:3],
                   model.get_layer('ssd_cls5conv2_bn').output_shape[1:3],
                   model.get_layer('ssd_cls6conv2_bn').output_shape[1:3]]
#encoder把ground truth labels
ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=True)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.

train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],# used to be augumentation
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	 38393
Number of images in the validation dataset:	  3530


In [16]:
initial_epoch   = 0
final_epoch     = 1000
# steps_per_epoch = 1 #1035#6000
steps_per_epoch = len(os.listdir('/root/OIDv4_ToolKit/OID/Dataset/train_all')) // batch_size
val_dataset_size = len(os.listdir('/root/OIDv4_ToolKit/OID/Dataset/validation_all'))

# Define model callbacks.
# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='/root/mobilenet_v2_ssdlite_keras/checkpoint.h5',
                                   monitor='val_loss',#used to be val_loss
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto')
# int(batch_size * steps_per_epoch)
                                  
#model_checkpoint.best = 

csv_logger = CSVLogger(filename='/root/mobilenet_v2_ssdlite_keras/MobileNetv2_ssdLite_training_log.csv',
                       separator=',',
                       append=True)

# learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule,
#                                                 verbose=1)

terminate_on_nan = TerminateOnNaN()

callbacks = [model_checkpoint,
             csv_logger,
             # learning_rate_scheduler,
             terminate_on_nan]

In [ ]:
# with tf.device('/device:GPU:0'):
with tf.device('/GPU:0'):
    history = model.fit(train_generator,
                        steps_per_epoch = steps_per_epoch,
                        epochs = final_epoch,
                        callbacks = callbacks,
                        validation_data = val_generator,
                        validation_steps = val_dataset_size//batch_size,
                        initial_epoch = initial_epoch,
                        verbose = 1)

2021-12-11 08:53:05.638156: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-11 08:53:05.639358: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999995000 Hz


Epoch 1/1000
Tensor("Neg:0", shape=(None, 1536), dtype=float32)


2021-12-11 08:53:18.199028: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2021-12-11 08:53:21.171501: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-12-11 08:53:21.823242: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


299/299 [==============================] - 3679s 12s/step - loss: 3.9142 - accuracy: 0.4309 - val_loss: 3.4870 - val_accuracy: 0.3325

Epoch 00001: val_loss improved from inf to 3.48705, saving model to /root/mobilenet_v2_ssdlite_keras/checkpoint.h5
Epoch 2/1000
299/299 [==============================] - 3598s 12s/step - loss: 3.8960 - accuracy: 0.4306 - val_loss: 3.5624 - val_accuracy: 0.3167

Epoch 00002: val_loss did not improve from 3.48705
Epoch 3/1000
299/299 [==============================] - 3549s 12s/step - loss: 3.8657 - accuracy: 0.4285 - val_loss: 3.5169 - val_accuracy: 0.3185

Epoch 00003: val_loss did not improve from 3.48705
Epoch 4/1000
299/299 [==============================] - 3532s 12s/step - loss: 3.8665 - accuracy: 0.4294 - val_loss: 3.5509 - val_accuracy: 0.3346

Epoch 00004: val_loss did not improve from 3.48705
Epoch 5/1000
299/299 [==============================] - 3566s 12s/step - loss: 3.8719 - accuracy: 0.4325 - val_loss: 3.7695 - val_accuracy: 0.3636

Epoch